# Veggie16

An exploration of how to adapt the VGG-16 CNN architecture, and modify the structure of the classification layer. Explores how to transfer learn and freeze the weights for the convolutional layers.

The developed Veggie16 is evaluated on the MNIST dataset.

This notebook is primarily inspired by the provided [CS284A CNN example](https://github.com/xhxuciedu/CS284A/blob/master/convolutional_neural_net.ipynb)

Make sure the runtime type of this notebook has a GPU.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms
from torchvision.models import vgg16

### Use GPU if available

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

### Define training hyperparameters

In [ ]:
# Hyper parameters
num_epochs = 25
num_classes = 10
batch_size = 100
learning_rate = 0.005

### Define a CNN based on VGG-16

In [ ]:
class Veggie16(nn.Module):
    """A model that adapts the VGG-16 architecture.

    This network applies transfer learning to learn the parameters
    of VGG-16, and freezes those layers of the model. The classification
    layer of the architecture is modified and will be retrained to 
    predict the desired number of output classes.
    """

    def __init__(self, num_classes):
        """Creates a Veggie16 network.

        Params:
            num_classes - The number of output classes to predict
        """
        super(Veggie16, self).__init__()
        architecture = vgg16(pretrained=True)
        self.features = architecture.features
        for param in self.features:
            param.requires_grad = False
        self.avgpool = architecture.avgpool
        self.classifier = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=num_classes, bias=True)
        )
    
    def forward(self, x):
        """Does a forward pass on an image x."""
        x_stack = torch.cat((x,x,x), 1) # MNIST is grayscale, but ImageNet is RGB
        out = self.features(x_stack)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out

model = Veggie16(10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
print(model)

### Load MNIST Data Set for Experiments

In [ ]:
data_dir = '../data'
train_dataset = torchvision.datasets.MNIST(root=data_dir, 
                                           train=True, 
                                           transform=transforms.Compose([
                                               transforms.Resize(256),
                                               transforms.CenterCrop(224),
                                               transforms.ToTensor()
                                           ]),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root=data_dir,
                                          train=False,
                                          transform=transforms.Compose([
                                               transforms.Pad(114),
                                               transforms.ToTensor()
                                           ]))


### Initialize `Dataloader` instances for reading data

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

### Train `Veggie16` model on MNIST Dataset

In [ ]:
num_steps = len(train_loader)
for epoch in range(1, num_epochs+1):
    for i, (images, labels) in enumerate(train_loader, start=1):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Step [{i}/{num_steps}], Loss: {loss.item():.6f}')

### Evaluate the effectiveness of `Veggie16`

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        total += labels.size(0)
        correct += (outputs == labels).sum().item()
    print(f'Test Accuracy of Veggie16: {(100*(correct/total)):.6f}')